## Loading neccessary libraries

In [1]:
import torch
from torch.utils.data import Dataset
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
# from transformers import AutoTokenizer, BertTokenizer, DistilBertModel
from transformers import (AutoTokenizer,
                          AutoModelForSequenceClassification,
                          AutoConfig,
                          TrainingArguments,
                          Trainer,
                          DataCollatorWithPadding)

## Loading the data

In [2]:
url_data = 'https://raw.githubusercontent.com/TeodorRusKvi/Tekstanalyse/main/git_NLP_data/'

df = pd.read_csv(url_data + 'new_df.csv')

In [3]:
y_liberal = pd.read_csv(url_data + 'y_liberal.csv')
y_data = pd.read_csv(url_data + 'y_data.csv')
X_text = pd.read_csv(url_data + 'X_text.csv')

In [4]:
X_text= X_text['Processed'].tolist()
y = y_liberal

In [5]:
text_train, text_test, y_train, y_test = train_test_split(
    X_text, 
    y, 
    test_size=0.2, 
    stratify=y,
    random_state=42)

## 4. Processing the data

In [6]:
from torch.utils.data import Dataset
import torch

class TextDataset(Dataset):
    def __init__(self, text, labels, tokenizer, max_length):
        self.tokenized_txt = tokenizer(
            text,
            max_length=max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        self.labels = torch.tensor(labels)
      
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            key: self.tokenized_txt[key][idx] for key in self.tokenized_txt
        }
        item['labels'] = self.labels[idx]
        return item

## 5. Initialize the model

In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

Roberta = 'roberta-base'
Bert = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(Roberta)
model = AutoModelForSequenceClassification.from_pretrained(Roberta, num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
dataset_train = TextDataset(
    text_train, 
    y_train.values, 
    tokenizer, 
    max_length=20
)

dataset_test = TextDataset(
    text_test, 
    y_test.values, 
    tokenizer, 
    max_length=20
)

from torch.utils.data import DataLoader

train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False)

In [9]:
dataset_test[9]

{'input_ids': tensor([    0, 13533, 18383,   102, 34377,  1469, 19473,  9937, 17343,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor([1])}

## 6. Training Setup

In [10]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1_result = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    balanced_acc = balanced_accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'balanced_accuracy': balanced_acc,
        'f1_score': f1_result
        }

In [11]:
# create a dir to store model checkpoints
my_dir = 'roberta_checkpoints'

### Using the Hugging Face Trainer

In [12]:
from torch.optim import AdamW
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader


optimizer = AdamW(model.parameters())
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 0.95**epoch)

# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# define training arguments
training_args = TrainingArguments(
    output_dir=my_dir,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    learning_rate=0.0001,
    lr_scheduler_type='cosine'
)

In [13]:
train_dataloader = DataLoader(dataset_train, batch_size=64, shuffle=True, collate_fn=data_collator)

for batch in train_dataloader:
    print(f"Input IDs shape: {batch['input_ids'].shape}")
    print(f"Labels shape: {batch['labels'].shape}")
    break

Input IDs shape: torch.Size([64, 20])
Labels shape: torch.Size([64, 1])


In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

c:\Users\bugat\anaconda3\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
# train the model
trainer.train()

  0%|          | 0/805 [00:00<?, ?it/s]

: 